In [0]:
# pacjages
import pandas as pd
import numpy as np
import os
import gzip
import scipy.stats
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
import seaborn as sns
import io
import requests
import json
from pandas.io.json import json_normalize
import networkx as nx

In [0]:
# data import
df_train   = pd.read_csv('data/train_tracking.csv.gz', compression='gzip')
df_test    = pd.read_csv('data/test_tracking.csv.gz',  compression='gzip')

# merge data
df = df_train.append(df_test)
df.shape

In [0]:
# duration variables
df['duration_no_days'] = df.duration.apply(lambda x: x[6:])
df['hours'] = df.duration_no_days.apply(lambda x: x[0:3]).astype('int')
df['min'] = df.duration_no_days.apply(lambda x: x[4:6]).astype('int')
df['secs'] = df.duration_no_days.apply(lambda x: x[7:9]).astype('int')
df['sec'] = df['hours']*360 + df['min']*60 + df['secs']
df.drop(['duration', 'duration_no_days','hours','min','secs'], axis=1, inplace=True)

In [0]:
# keep relevant data
df = df[["sid", "type", "sec"]]

# sort and prepare data
df = df.sort_values(["sid", "sec"])
df = df[["sid", "type"]]
df["type2"] = df.groupby("sid").shift(-1)
df = df[-df.type2.isnull()]

In [0]:
# placeholder
feats = pd.DataFrame(newres)

In [0]:
# graph features function
def compute_graph_features(id):

    tmp_df = df[df.sid == id][["type", "type2"]]
    g = nx.from_pandas_edgelist(tmp_df, "type", 'type2')

    if tmp_df.shape[0] > 5:
                    
        feats = [id,
        nx.diameter(g),
        np.mean(list(nx.eccentricity(g).values())),
        len(nx.periphery(g)),
        nx.radius(g),
        len(nx.center(g)),
        len(nx.nodes(g)),
        nx.density(g),
        nx.transitivity(g),
        nx.number_of_edges(g),
        nx.number_of_selfloops(g),
        nx.node_connectivity(g),
        np.mean(list(nx.average_degree_connectivity(g).values())),
        np.mean(list(nx.average_neighbor_degree(g).values())),
        np.mean(list(nx.closeness_centrality(g).values())),
        np.mean(list(nx.betweenness_centrality(g).values())),
        np.mean(list(nx.katz_centrality(g).values())),
        np.mean(list(nx.pagerank(g).values())),
        np.mean(list(nx.eigenvector_centrality(g).values()))]

    else:
      
      feats = [id, np.zeros(17)]
      
    return feats

In [0]:
# create pool
from multiprocessing import Pool
pool = Pool(22)

In [0]:
# compute features
for i in range(0, 1):  
 
  if i%50 == 0:
    print(i)
 
  try:
    newres = pool.map(compute_graph_features, ids[(0+100*i):(100+i*100)])
  except:
    pass
  
  feats = feats.append(pd.DataFrame(newres))

pool.close()

In [0]:
# export
feats.columns = ["sid", "g1", "g2", "g3", "g4", "g5", "g6", "g7", "g8", "g9", "g10",
                 "g11", "g12", "g13", "g14", "g15", "g16", "g17", "g18"]
feats.to_csv("data/ggg3.csv", index = False)